In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds


2022-12-14 14:51:01.728097: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 14:51:01.958863: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-14 14:51:02.016643: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-14 14:51:02.016659: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [104]:
class config:
    DATASET_PATH="imdb_dataset"

    BATCH_SIZE=1024
    BUFFER_SIZE=BATCH_SIZE*2

    VOCAB_SIZE=20000
    MAX_SEQ_LEN=50

    EPOCHS=15

    LR=1e-4
    autotune=tf.data.AUTOTUNE
    OUTPUT_PATH='output'
    RNN_MODEL_PATH=os.path.join(OUTPUT_PATH,'rnn_model')
    LSTM_MODEL_PATH=os.path.join(OUTPUT_PATH,'lstm_model')
    TEXT_VEC_PATH=os.path.join(OUTPUT_PATH,'text_vec')

    RNN_PLOT=os.path.join(OUTPUT_PATH,'rnn_plot.png')
    LSTM_PLOT=os.path.join(OUTPUT_PATH,'lstm_plot.png')
    URL = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
    FILE_NAME='aclImdb'
    DATASET_NAME='aclImdb_v1.tar.gz'
    EMBEDDING_SIZE=100  
config=config()

In [100]:
import shutil
import os

def fetch_dataset(url,dataName,fileName):
    dataset=tf.keras.utils.get_file('aclImdb_v1.tar.gz',url,untar=True,cache_dir='.',cache_subdir='')
    dataset_dir=os.path.join(os.path.dirname(dataset),fileName)
    train_dir=os.path.join(dataset_dir,'train')
    shutil.rmtree(os.path.join(train_dir,'unsup'))
    rawTrainDS=tf.keras.utils.text_dataset_from_directory(train_dir,batch_size=config.BATCH_SIZE,validation_split=0.2,subset='training',seed=42)
    rawValDS=tf.keras.utils.text_dataset_from_directory(train_dir,batch_size=config.BATCH_SIZE,validation_split=0.2,subset='validation',seed=42)
    rawTestDS=tf.keras.utils.text_dataset_from_directory(config.FILE_NAME+'/test',batch_size=config.BATCH_SIZE)
    
    rawTrainDS=rawTrainDS.cache().prefetch(config.autotune)
    rawValDS=rawValDS.cache().prefetch(config.autotune)
    rawTestDS=rawTestDS.cache().prefetch(config.autotune)
    return (rawTrainDS,rawValDS,rawTestDS)
with tf.device('/cpu:0'):
    rawTrainDS,rawValDS,rawTestDS=fetch_dataset(url=config.URL,dataName=config.DATASET_NAME,fileName=config.FILE_NAME)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [101]:
for i,j in rawTrainDS.take(1):
    print(i[:1])

tf.Tensor([b"I married a Japanese woman 14 years ago. We're still together.<br /><br />However in the 1950's it would never have been as easy.<br /><br />Life in the military had been mined for action, drama, and comedy for years by this point. Mined to death. The mixed relationships gave it new ground to cover. This is old hat today, but then...? Marrying an Asian back then meant you either owed somebody something or you were a freak of some sort. This touched on both possibilities along with the third. Maybe it IS love? <br /><br />Brando did his usual good job. Garner did a better job than he usually does. He's good, but this showed how good he could be. Umecki-chan had a helluva debut here and while I think she earned her statue, she didn't really stretch. It was a role that no one who hadn't been overseas would have recognized and the newness was the corker.<br /><br />The real scene stealer was Red Buttons. Red was the best thing in this film. Bank on it. And the Japanese lifesty

2022-12-15 16:04:32.354145: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [102]:
encoder=tf.keras.layers.TextVectorization(max_tokens=config.VOCAB_SIZE)
encoder.adapt(rawTrainDS.map(lambda text,label:text))

In [146]:
for i,j in rawTrainDS.take(1):
    example=i[:10]

In [181]:
class build_RNN(tf.keras.Model):
    def __init__(self,rnn_cells,vocab_size,emb_dim):
        super().__init__(self)
        self.emb=tf.keras.layers.Embedding(vocab_size,emb_dim)
        self.lstm=tf.keras.layers.LSTM(rnn_cells,return_sequences=False,return_state=True)
        self.fc_1=tf.keras.layers.Dense(1000,kernel_initializer=tf.initializers.glorot_normal())
        self.fc_out=tf.keras.layers.Dense(1,activation='sigmoid') 
    def call(self,inputs,states=None,training=False,return_state=False):
        emb=self.emb(inputs,training=training)
        if not states:
            states=self.lstm.get_initial_state(emb)
        x,*states=self.lstm(emb,initial_state=states,training=training)
        x=self.fc_1(x,training=training)
        x=self.fc_out(x,training=training)
        if return_state:
            return (x,states)
        else:
            return x


In [182]:
model=build_RNN(rnn_cells=500,vocab_size=config.VOCAB_SIZE,emb_dim=config.EMBEDDING_SIZE)

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.50031555],
       [0.50031555],
       [0.50031555],
       [0.50031555],
       [0.50031555],
       [0.50031555],
       [0.50031555],
       [0.5012726 ],
       [0.50031555],
       [0.50031555]], dtype=float32)>

In [185]:
model.summary()

Model: "build_rnn_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    multiple                  2000000   
                                                                 
 lstm_12 (LSTM)              multiple                  1202000   
                                                                 
 dense_24 (Dense)            multiple                  501000    
                                                                 
 dense_25 (Dense)            multiple                  1001      
                                                                 
Total params: 3,704,001
Trainable params: 3,704,001
Non-trainable params: 0
_________________________________________________________________
